## Linear Models, part II: Logistic Regression

1. Find and read into a DataFrame a suitable dataset. You may use the global social indictors data from the example here. You may need to combine files, as shown here.

2. Identify a dependent variable to explain. Create a binary variable of this measure, if needed. Explain why you chose this variable or recoded in the way you did.

3. Build a model to explain the DV. You can use the odds (anti-log of the coefficients) or the marginal effects to test for the unique effects of each predictor.

4. Explain the results.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sb
import math

In [6]:
GlobalIndicators1 = pd.read_excel('http://data.shortell.nyc/files/HumanDevelopment.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators2 = pd.read_excel('http://data.shortell.nyc/files/GenderDevelopment.xlsx', index_col='Country', na_values=['NA'])
GlobalIndicators3 = pd.read_excel('http://data.shortell.nyc/files/GenderInequality.xlsx', index_col='Country', na_values=['NA'])

In [28]:
GlobalIndicatorsTotal = pd.concat([GlobalIndicators1, GlobalIndicators2, GlobalIndicators3], axis=1)
GlobalIndicatorsTotal.columns

Index(['HDI Rank', 'Human Development Index (HDI)', 'Life Expectancy at Birth',
       'Expected Years of Education', 'Mean Years of Education',
       'Gross National Income (GNI) per Capita',
       'GNI per Capita Rank Minus HDI Rank', 'GDI Rank',
       'Gender Development Index (GDI)', 'Human Development Index (Female)',
       'Human Development Index (Male)', 'Life Expectancy at Birth (Female)',
       'Life Expectancy at Birth (Male)',
       'Expected Years of Education (Female)',
       'Expected Years of Education (Male)',
       'Mean Years of Education (Female)', 'Mean Years of Education (Male)',
       'Estimated Gross National Income per Capita (Female)',
       'Estimated Gross National Income per Capita (Male)', 'GII Rank',
       'Gender Inequality Index (GII)', 'Maternal Mortality Ratio',
       'Adolescent Birth Rate', 'Percent Representation in Parliament',
       'Population with Secondary Education (Female)',
       'Population with Secondary Education (Male)',
 

In [29]:

# The GDI is the ratio of the HDIs calculated separately for females and males using the same methodology as in the HDI. It is a direct measure of gender gap, showing the female HDI as a percentage of the male HDI. 
GlobalIndicatorsTotal['Gender Development Index (GDI)'].describe()

count    161.000000
mean       0.930714
std        0.072619
min        0.600000
25%        0.896000
50%        0.950000
75%        0.980000
max        1.030000
Name: Gender Development Index (GDI), dtype: float64

In [30]:
GlobalIndicatorsTotal['GII Binary'] = 0
# Sets all countries with a gender gap within one standard of deviation (.930714 - .072619) as true 
GlobalIndicatorsTotal.loc[GlobalIndicatorsTotal['Gender Development Index (GDI)'] < 0.8581, ['GII Binary']] = 1
GlobalIndicatorsTotal['GII Binary']

Country
Norway                      0
Australia                   0
Switzerland                 0
Denmark                     0
Netherlands                 0
                           ..
Burundi                     0
Chad                        1
Eritrea                     0
Central African Republic    1
Niger                       1
Name: GII Binary, Length: 188, dtype: int64

In [31]:
Y = GlobalIndicatorsTotal['GII Binary']
X = GlobalIndicatorsTotal[['Percent Representation in Parliament', 'Population with Secondary Education (Female)', 'Labour Force Participation Rate (Female)', 'Estimated Gross National Income per Capita (Female)', 'Gross National Income (GNI) per Capita']]
model0 = sm.Logit(Y, X, missing='drop').fit()
print(model0.summary())

Optimization terminated successfully.
         Current function value: 0.165027
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:             GII Binary   No. Observations:                  156
Model:                          Logit   Df Residuals:                      151
Method:                           MLE   Df Model:                            4
Date:                Wed, 02 Nov 2022   Pseudo R-squ.:                  0.5943
Time:                        14:03:24   Log-Likelihood:                -25.744
converged:                       True   LL-Null:                       -63.464
Covariance Type:            nonrobust   LLR p-value:                 1.608e-15
                                                          coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Percent Representation in

In [ ]:
print(1/math.exp(model0.params[0]), 1/math.exp(model0.params[1]), math.exp(model0.params[2]), 1/math.exp(model0.params[3]), math.exp(1000*model0.params[4]))

In [33]:
model0_marginals = model0.get_margeff()
print(model0_marginals.summary())

        Logit Marginal Effects       
Dep. Variable:             GII Binary
Method:                          dydx
At:                           overall
                                                         dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament                   -0.0016      0.002     -1.032      0.302      -0.005       0.001
Population with Secondary Education (Female)           -0.0011      0.001     -0.975      0.330      -0.003       0.001
Labour Force Participation Rate (Female)                0.0019      0.001      2.729      0.006       0.001       0.003
Estimated Gross National Income per Capita (Female)    -0.0001   2.48e-05     -5.990      0.000      -0.000   -9.97e-05
Gross National Income (GNI) per Capita               5.406e-05   9.19e-06      5.879      0.000     3.6e-05    7.21e-05


In [34]:
model0_pred = model0.pred_table()
print(model0_pred) # Correct predictions are on the diagonal of the 2d array.

[[129.   5.]
 [ 10.  12.]]


In [36]:
correct_i = 129 / (129 + 5) # The proportion of correct predictions of 0.
correct_j = 12 / (12 + 10) # The proportion of correct predictions of 1.
print(correct_i, correct_j)

0.9626865671641791 0.5454545454545454


I believe this model is showing that it does a very good job of predicting which countries have a low Gender Development Index (GDI) based on these countries' - 'Percent Representation in Parliament', 'Population with Secondary Education (Female)', 'Labour Force Participation Rate (Female)', 'Estimated Gross National Income per Capita (Female)', and 'Gross National Income (GNI) per Capita.' 

But inversely the model basically does a coin flip's chance of predicting which countries have a high GDI based on the same indicators.

This makes some sense considering that only 25% of the countries' GDI are below 89%. Yet there is still high variability within the tested X values within those countries.